In [1]:
from lxml import etree
from io import StringIO, BytesIO
import json
import time
import re

In [ ]:
start_time = time.time()

titles = []
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    if event == 'start' and 'title' in elem.tag:
        if elem.text is not None:
            titles.append(elem.text.lower())
#             titles.append({'title': elem.text.lower(), 'redirect': ''})
#         titles.append((elem.text,))
    elif event == 'start' and 'redirect' in elem.tag:
        titles[-1] = elem.attrib['title'].lower()
#         titles[-1]['redirect'] = elem.attrib['title'].lower()
#         titles[-1] = titles[-1] + (elem.attrib['title'],)
    elem.clear()
    
print('Titles: ', len(titles))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

with open('./wiki-dump/titles', 'w') as outfile:
    json.dump(titles, outfile, indent=2)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

titles = []
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    if event == 'start' and 'page' in elem.tag:
        f = elem.find('{http://www.mediawiki.org/xml/export-0.10/}redirect')
        if f is not None:
            titles.append(f.attrib['title'].lower())
        else:
            f = elem.find('{http://www.mediawiki.org/xml/export-0.10/}title')
            if f is not None and f.text is not None:
                titles.append(f.text.lower())
    elem.clear()
    
print('Titles: ', len(titles))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
count = 1000
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    if event == 'start' and 'page' in elem.tag:
        f = elem.find('{http://www.mediawiki.org/xml/export-0.10/}redirect')
        if f is not None:
            print(f.attrib['title'])
        else:
            f = elem.find('{http://www.mediawiki.org/xml/export-0.10/}title')
            print(f.text)
    if count <= 0:
        break
    count -= 1
    elem.clear()
        

In [ ]:
count = 170
text = ''
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    if event == 'start' and 'text' in elem.tag:
        count -= 1
        if count == 0:
            text = elem.text
            break

# m = re.findall('\[\[(.*?)]]', text)
m = re.findall('\[\[([^][]+)]]', text)
print(len(m))
print(m)

for i, s in enumerate(m):
    m[i] = s.split('|')[-1]

print(m)


In [ ]:
print(text)


In [2]:
vocabulary = []
alternative_keyphrases = set()

start_time = time.time()

# Extract keyphrases from articles
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    # Extract titles of pages
    if event == 'start' and 'title' in elem.tag:
        if elem.text is not None:
            vocabulary.append(elem.text.lower())

    elif event == 'start' and 'redirect' in elem.tag:
        vocabulary[-1] = elem.attrib['title'].lower()
    
    # Extract keyphrases in other morphological form
    elif event == 'start' and 'text' in elem.tag:
        text = elem.text
        if text is not None:
            keyphrases = re.findall('\[\[([^][]+)]]', text)
            for key in keyphrases:
                alternative_keyphrases.add(key.split('|')[-1].lower())
        
    elem.clear()

vocabulary = alternative_keyphrases.union(set(vocabulary))
print('Vocabulary: ', len(vocabulary))

print("--- %s seconds ---" % (time.time() - start_time))


Vocabulary:  894503
--- 18.574403524398804 seconds ---
